## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Library

In [ ]:
import os
import pickle
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

## Validasi

In [ ]:
# Path hasil ekstraksi fitur
path_output = "/content/drive/MyDrive/STKI/DatasetFinal/model"
train_file = os.path.join(path_output, "fitur_vggface.pkl")
val_file = os.path.join(path_output, "fitur_val_vggface.pkl")

In [ ]:
# Load fitur dan label untuk train
if not os.path.exists(train_file):
    raise FileNotFoundError(f"Hasil ekstraksi train tidak ditemukan di {train_file}")
if not os.path.exists(val_file):
    raise FileNotFoundError(f"Hasil ekstraksi val tidak ditemukan di {val_file}")

with open(train_file, "rb") as f:
    train_data = pickle.load(f)
    train_features = np.array(train_data["features"])
    train_labels = np.array(train_data["labels"])

with open(val_file, "rb") as f:
    val_data = pickle.load(f)
    val_features = np.array(val_data["features"])
    val_labels = np.array(val_data["labels"])

In [ ]:
# Label encoding
label_map = {"Drowsy": 0, "Non Drowsy": 1}
train_labels_encoded = np.array([label_map[label] for label in train_labels])
val_labels_encoded = np.array([label_map[label] for label in val_labels])

# One-hot encoding untuk label
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=2)
val_labels_categorical = to_categorical(val_labels_encoded, num_classes=2)

In [ ]:
# Bangun model FCNN
print("Membangun model FCNN...")
model = Sequential([
    Dense(256, activation='relu', input_shape=(train_features.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Latih model
print("Melatih model FCNN...")
history = model.fit(
    train_features, train_labels_categorical,
    epochs=20,
    batch_size=32,
    validation_data=(val_features, val_labels_categorical)
)

# Simpan model
model_save_path = os.path.join(path_output, "fcnn_model.h5")
model.save(model_save_path)
print(f"Model disimpan di {model_save_path}")

Membangun model FCNN...
Melatih model FCNN...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.9382 - loss: 0.1373 - val_accuracy: 0.9976 - val_loss: 0.0065
Epoch 2/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.9964 - loss: 0.0108 - val_accuracy: 0.9981 - val_loss: 0.0043
Epoch 3/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 40s 22ms/step - accuracy: 0.9986 - loss: 0.0050 - val_accuracy: 0.9986 - val_loss: 0.0036
Epoch 4/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9984 - loss: 0.0049 - val_accuracy: 0.9986 - val_loss: 0.0045
Epoch 5/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.9987 - loss: 0.0032 - val_accuracy: 0.9987 - val_loss: 0.0044
Epoch 6/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.9994 - loss: 0.0021 - val_accuracy: 0.9987 - val_loss: 0.0057
Epoch 7/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9995 - loss: 0.0014 - val_accuracy: 0.9986 - val_loss: 0.0038
Epoch 8/20
914/914 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.9984 - loss: 0.0049 - 

Model disimpan di /content/drive/MyDrive/STKI/DatasetFinal/model/fcnn_model.h5


In [ ]:
# Evaluasi model
print("Evaluasi model pada data validasi...")
predictions = model.predict(val_features)
predicted_labels = np.argmax(predictions, axis=1)

print("Classification Report:")
print(classification_report(val_labels_encoded, predicted_labels, target_names=["Drowsy", "Non Drowsy"]))

Evaluasi model pada data validasi...
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Classification Report:
              precision    recall  f1-score   support

      Drowsy       1.00      1.00      1.00      3344
  Non Drowsy       1.00      1.00      1.00      2905

    accuracy                           1.00      6249
   macro avg       1.00      1.00      1.00      6249
weighted avg       1.00      1.00      1.00      6249

